In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# 📁 Load dataset from folders
def load_images_from_folder(folder_path, image_size=(48, 48), grayscale=True):
    images = []
    labels = []

    for class_label in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_label)
        if not os.path.isdir(class_path):
            continue

        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            if grayscale:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            else:
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)

            if img is None:
                print(f"Skipping: {img_path}")
                continue

            img_resized = cv2.resize(img, image_size)
            images.append(img_resized)
            labels.append(class_label)

    return np.array(images), np.array(labels)

X, y = load_images_from_folder(r'C:\Users\ANUPAM MANDAL\OneDrive\Documents\major_pro\Final_Dataset', image_size=(48, 48))
print(f"✅ Loaded {len(X)} images from dataset with {len(np.unique(y))} emotion classes.")

X = X.astype('float32') / 255.0
X = X.reshape(-1, 48, 48, 1)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

def create_cnn(input_shape=(48, 48, 1), num_classes=5):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_model = create_cnn(input_shape=(48, 48, 1), num_classes=len(np.unique(y)))
cnn_model.summary()

history = cnn_model.fit(X_train, y_train, epochs=60, batch_size=32, validation_split=0.2)

test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {test_accuracy * 100:.2f}%")

y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

class_names = label_encoder.classes_

report = classification_report(y_true, y_pred_classes, target_names=class_names)
print("📊 Classification Report:\n")
print(report)

precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred_classes, zero_division=0)
x = np.arange(len(class_names))

plt.figure(figsize=(10, 6))
plt.bar(x - 0.25, precision, width=0.25, label='Precision', color='skyblue')
plt.bar(x, recall, width=0.25, label='Recall', color='lightgreen')
plt.bar(x + 0.25, f1_score, width=0.25, label='F1 Score', color='salmon')

plt.xticks(x, class_names, rotation=45)
plt.xlabel('Emotion Class')
plt.ylabel('Score')
plt.title('Precision, Recall, and F1 Score per Emotion Class')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss', color='red')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("Model Loss Over Epochs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

def predict_emotion(model, image_path):
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_color = cv2.imread(image_path)

    if img_gray is None or img_color is None:
        print("❌ Error: Image not found or unreadable.")
        return

    img_resized = cv2.resize(img_gray, (48, 48)) / 255.0
    img_reshaped = img_resized.reshape(1, 48, 48, 1)

    prediction = model.predict(img_reshaped)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])[0]

    img_color_resized = cv2.resize(img_color, (200, 200))
    img_rgb = cv2.cvtColor(img_color_resized, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(4, 4))
    plt.imshow(img_rgb)
    plt.title(f"Predicted Emotion: {predicted_label}", fontsize=14, color='green')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return predicted_label

test_image_path = r'C:\Users\ANUPAM MANDAL\OneDrive\Documents\major_pro\images\61.jpg'
predicted_emotion = predict_emotion(cnn_model, test_image_path)
print(f"🧠 Predicted Emotion: {predicted_emotion}")

